In [0]:
!pip install mxnet-cu100
!pip install d2lzh


In [3]:
%cd /content/drive/My Drive/Colab Notebooks/dog-breed-standford
!pwd
!ls

/content/drive/My Drive/Colab Notebooks/dog-breed-standford
/content/drive/My Drive/Colab Notebooks/dog-breed-standford
content      r152i3.params		train.png
ids_synsets  sample_submission.csv	train_r152i3.nd
Images	     sample_submission.csv.zip	vision.stanford.edu


In [0]:
from mxnet import gluon
from mxnet import image
import numpy as np
from mxnet import nd
import math
import os
import shutil
from collections import Counter

In [0]:
data_dir = './data'
label_file = 'labels.csv'
train_dir = 'train'
test_dir = 'test'
input_dir = 'train_valid_test'
batch_size = 128
valid_ratio = 0.1

load **data**  and image transform 

In [0]:
def transform_train(data, label):
    im1 = image.imresize(data.astype('float32') / 255, 224, 224)
    im2 = image.imresize(data.astype('float32') / 255, 299, 299)
    auglist1 = image.CreateAugmenter(data_shape=(3, 224, 224), resize=0, 
                        rand_crop=False, rand_resize=False, rand_mirror=True,
                        mean=np.array([0.485, 0.456, 0.406]), std=np.array([0.229, 0.224, 0.225]), 
                        brightness=0, contrast=0, 
                        saturation=0, hue=0, 
                        pca_noise=0, rand_gray=0, inter_method=2)
    auglist2 = image.CreateAugmenter(data_shape=(3, 299, 299), resize=0, 
                        rand_crop=False, rand_resize=False, rand_mirror=True,
                        mean=np.array([0.485, 0.456, 0.406]), std=np.array([0.229, 0.224, 0.225]), 
                        brightness=0, contrast=0, 
                        saturation=0, hue=0, 
                        pca_noise=0, rand_gray=0, inter_method=2)
    for aug in auglist1:
        im1 = aug(im1)
    for aug in auglist2:
        im2 = aug(im2)
    # 将数据格式从"高*宽*通道"改为"通道*高*宽"。
    im1 = nd.transpose(im1, (2,0,1))
    im2 = nd.transpose(im2, (2,0,1))
    return (im1,im2, nd.array([label]).asscalar().astype('float32'))

def transform_test(data, label):
    im1 = image.imresize(data.astype('float32') / 255, 224, 224)
    im2 = image.imresize(data.astype('float32') / 255, 299, 299)
    auglist1 = image.CreateAugmenter(data_shape=(3, 224, 224),
                        mean=np.array([0.485, 0.456, 0.406]), 
                        std=np.array([0.229, 0.224, 0.225]))
    auglist2 = image.CreateAugmenter(data_shape=(3, 299, 299),
                        mean=np.array([0.485, 0.456, 0.406]), 
                        std=np.array([0.229, 0.224, 0.225]))
    for aug in auglist1:
        im1 = aug(im1)
    for aug in auglist2:
        im2 = aug(im2)
    # 将数据格式从"高*宽*通道"改为"通道*高*宽"。
    im1 = nd.transpose(im1, (2,0,1))
    im2 = nd.transpose(im2, (2,0,1))
    return (im1,im2, nd.array([label]).asscalar().astype('float32'))

In [0]:
batch_size = 32

train_ds = gluon.data.vision.ImageFolderDataset('/content/drive/My Drive/Colab Notebooks/dog-breed-standford/Images', flag=1,transform=transform_train)
#valid_ds = gluon.data.vision.ImageFolderDataset('/content/drive/My Drive/Colab Notebooks/dog-breed/train_valid_test/valid', flag=1, transform=transform_test)
#train_valid_ds = gluon.data.vision.ImageFolderDataset('/content/drive/My Drive/Colab Notebooks/dog-breed/train_valid_test/valid', flag=1, transform=transform_train)
test_ds = gluon.data.vision.ImageFolderDataset('/content/drive/My Drive/Colab Notebooks/dog-breed/train_valid_test/test', flag=1, transform=transform_test)

loader = gluon.data.DataLoader
train_data = loader(train_ds, batch_size, shuffle=True, last_batch='keep')
#valid_data = loader(valid_ds, batch_size, shuffle=True, last_batch='keep')
#train_valid_data = loader(train_valid_ds, batch_size, shuffle=True, last_batch='keep')

 **define network and fine tuning**

In [0]:
from mxnet import init
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision

In [0]:
class  ConcatNet(nn.HybridBlock):
    def __init__(self,net1,net2,**kwargs):
        super(ConcatNet,self).__init__(**kwargs)
        self.net1 = nn.HybridSequential()
        self.net1.add(net1)
        self.net1.add(nn.GlobalAvgPool2D())
        self.net2 = nn.HybridSequential()
        self.net2.add(net2)
        self.net2.add(nn.GlobalAvgPool2D())
    def hybrid_forward(self,F,x1,x2):
        return F.concat(*[self.net1(x1),self.net2(x2)])

In [0]:
class  OneNet(nn.HybridBlock):
    def __init__(self,features,output,**kwargs):
        super(OneNet,self).__init__(**kwargs)
        self.features = features
        self.output = output
    def hybrid_forward(self,F,x1,x2):
        return self.output(self.features(x1,x2))

In [0]:
class Net():
    def __init__(self,ctx,nameparams=None):
        inception = vision.inception_v3(pretrained=True,ctx=ctx).features
        resnet = vision.resnet152_v1(pretrained=True,ctx=ctx).features
        self.features = ConcatNet(resnet,inception)
        self.output = self.__get_output(ctx,nameparams)
        self.net = OneNet(self.features,self.output)
    def __get_output(self,ctx,ParamsName=None):
        net = nn.HybridSequential("output")
        with net.name_scope():
            net.add(nn.Dense(256,activation='relu'))
            net.add(nn.Dropout(.5))
            net.add(nn.Dense(120))
        if ParamsName is not None:
            net.collect_params().load(ParamsName,ctx)
        else:
            net.initialize(init = init.Xavier(),ctx=ctx)
        return net

train and extract feature

In [0]:
from tqdm import tqdm
import datetime
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from mxnet import autograd
import mxnet as mx
import pickle

In [107]:
ctx=mx.gpu()
net = Net(ctx).features
net.hybridize()

def SaveNd(data,net,name):
    x =[]
    y =[]
    print('提取特征 %s' % name)
    for fear1,fear2,label in tqdm(data):
        fear1 = fear1.as_in_context(mx.gpu())
        fear2 = fear2.as_in_context(mx.gpu())
        out = net(fear1,fear2).as_in_context(mx.cpu())
        x.append(out)
        y.append(label)
    x = nd.concat(*x,dim=0)
    y = nd.concat(*y,dim=0)
    print('保存特征 %s' % name)
    nd.save(name,[x,y])

SaveNd(train_data,net,'train_r152i3.nd')
#SaveNd(valid_data,net,'valid_r152i3.nd')
#SaveNd(train_valid_data,net,'input_r152i3.nd')



  0%|          | 0/644 [00:00<?, ?it/s]

提取特征 train_r152i3.nd




  0%|          | 1/644 [00:08<1:35:46,  8.94s/it]

  0%|          | 2/644 [00:16<1:32:37,  8.66s/it]

  0%|          | 3/644 [00:24<1:30:24,  8.46s/it]

  1%|          | 4/644 [00:33<1:29:05,  8.35s/it]

  1%|          | 5/644 [00:41<1:28:54,  8.35s/it]

  1%|          | 6/644 [00:51<1:32:51,  8.73s/it]

  1%|          | 7/644 [01:00<1:36:36,  9.10s/it]

  1%|          | 8/644 [01:10<1:36:17,  9.08s/it]

  1%|▏         | 9/644 [01:20<1:39:24,  9.39s/it]

  2%|▏         | 10/644 [01:27<1:33:03,  8.81s/it]

  2%|▏         | 11/644 [01:33<1:24:20,  7.99s/it]

  2%|▏         | 12/644 [01:41<1:23:47,  7.95s/it]

  2%|▏         | 13/644 [01:51<1:30:05,  8.57s/it]

  2%|▏         | 14/644 [02:00<1:30:11,  8.59s/it]

  2%|▏         | 15/644 [02:06<1:21:46,  7.80s/it]

  2%|▏         | 16/644 [02:16<1:28:23,  8.45s/it]

  3%|▎         | 17/644 [02:27<1:36:46,  9.26s/it]

  3%|▎         | 18/644 [02:32<1:23:34,  8.01s/it]

  3%|▎         | 19/644 [02:41<1:27:33,  8.41s/it]

  3%|▎         | 20

保存特征 train_r152i3.nd


In [0]:
ids = ids = sorted(os.listdir(os.path.join('/content/drive/My Drive/Colab Notebooks/dog-breed/test')))
synsets = train_ds.synsets
f = open('ids_synsets','wb')
pickle.dump([ids,synsets],f)
f.close()

In [0]:
num_epochs = 300
batch_size = 128
learning_rate = 1e-4
weight_decay = 1e-4
pngname='train.png'
modelparams='r152i3.params'

In [0]:
train_nd = nd.load('train_r152i3.nd')
#valid_nd = nd.load('valid_r152i3.nd')
#input_nd = nd.load('input_r152i3.nd')
f = open('ids_synsets','rb')
ids_synsets = pickle.load(f)
f.close()

train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(train_nd[0],train_nd[1]), batch_size=batch_size,shuffle=True)
#valid_data = gluon.data.DataLoader(gluon.data.ArrayDataset(valid_nd[0],valid_nd[1]), batch_size=batch_size,shuffle=True)
#input_data = gluon.data.DataLoader(gluon.data.ArrayDataset(input_nd[0],input_nd[1]), batch_size=batch_size,shuffle=True)

In [0]:
def get_loss(data, net, ctx):
    loss = 0.0
    for feas, label in data:
        label = label.as_in_context(ctx)
        output = net(feas.as_in_context(ctx))
        cross_entropy = softmax_cross_entropy(output, label)
        loss += nd.mean(cross_entropy).asscalar()
    return loss / len(data)

def train(net, train_data, valid_data, num_epochs, lr, wd, ctx):
    trainer = gluon.Trainer(
        net.collect_params(), 'adam', {'learning_rate': lr, 'wd': wd})
    train_loss = []
    if valid_data is not None:
        test_loss = []
    
    prev_time = datetime.datetime.now()
    for epoch in range(num_epochs):
        _loss = 0.
        for data, label in train_data:
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            _loss += nd.mean(loss).asscalar()
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        __loss = _loss/len(train_data)
        train_loss.append(__loss)
        
        if valid_data is not None:  
            valid_loss = get_loss(valid_data, net, ctx)
            epoch_str = ("Epoch %d. Train loss: %f, Valid loss %f, "
                         % (epoch,__loss , valid_loss))
            test_loss.append(valid_loss)
        else:
            epoch_str = ("Epoch %d. Train loss: %f, "
                         % (epoch, __loss))
            
        prev_time = cur_time
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))
        

    plt.plot(train_loss, 'r')
    if valid_data is not None: 
        plt.plot(test_loss, 'g')
    plt.legend(['Train_Loss', 'Test_Loss'], loc=2)


    plt.savefig(pngname, dpi=1000)
    net.collect_params().save(modelparams)

In [25]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
ctx = mx.gpu()
net = Net(ctx).output
net.hybridize()

train(net, train_data,None, num_epochs, learning_rate, weight_decay, ctx)

Epoch 0. Train loss: 3.122290, Time 00:00:00, lr 0.0001
Epoch 1. Train loss: 1.012736, Time 00:00:00, lr 0.0001
Epoch 2. Train loss: 0.622263, Time 00:00:00, lr 0.0001
Epoch 3. Train loss: 0.490362, Time 00:00:00, lr 0.0001
Epoch 4. Train loss: 0.420578, Time 00:00:00, lr 0.0001
Epoch 5. Train loss: 0.366289, Time 00:00:00, lr 0.0001
Epoch 6. Train loss: 0.337918, Time 00:00:00, lr 0.0001
Epoch 7. Train loss: 0.312941, Time 00:00:00, lr 0.0001
Epoch 8. Train loss: 0.297313, Time 00:00:00, lr 0.0001
Epoch 9. Train loss: 0.279169, Time 00:00:00, lr 0.0001
Epoch 10. Train loss: 0.267185, Time 00:00:00, lr 0.0001
Epoch 11. Train loss: 0.254837, Time 00:00:00, lr 0.0001
Epoch 12. Train loss: 0.238026, Time 00:00:00, lr 0.0001
Epoch 13. Train loss: 0.229642, Time 00:00:00, lr 0.0001
Epoch 14. Train loss: 0.222299, Time 00:00:00, lr 0.0001
Epoch 15. Train loss: 0.208899, Time 00:00:00, lr 0.0001
Epoch 16. Train loss: 0.203869, Time 00:00:00, lr 0.0001
Epoch 17. Train loss: 0.196182, Time 00:0

In [0]:
netparams = 'r152i3.params'
csvname = 'kaggle.csv'
ids_synsets_name = 'ids_synsets'
f = open(ids_synsets_name,'rb')
ids_synsets = pickle.load(f)
f.close()

In [0]:
#test_ds = vision.ImageFolderDataset(input_str + test_dir, flag=1,transform=transform_test)
loader = gluon.data.DataLoader
test_data = loader(test_ds, batch_size, shuffle=False, last_batch='keep')
def SaveTest(test_data,net,ctx,name,ids,synsets):
    outputs = []
    for data1,data2, label in tqdm(test_data):
        data1 =data1.as_in_context(ctx)
        data2 =data2.as_in_context(ctx)
        output = nd.softmax(net(data1,data2))
        outputs.extend(output.asnumpy())
    with open(name, 'w') as f:
        f.write('id,' + ','.join(synsets) + '\n')
        for i, output in zip(ids, outputs):
            f.write(i.split('.')[0] + ',' + ','.join(
                [str(num) for num in output]) + '\n')

In [0]:
net = Net(mx.gpu(),netparams).net
net.hybridize()
SaveTest(test_data,net,mx.gpu(),csvname,ids_synsets[0],ids_synsets[1])

 83%|████████▎ | 67/81 [1:30:24<16:18, 69.86s/it]

In [2]:
!kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 6, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
！7z